## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
* Build video processing pipeline.

---
## 1. Camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## 2. Distortion correction of raw images

In [2]:
def dist_corr(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## 3. Create thresholded binary image

In [3]:
def sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    
    S = image[:,:,2]
    oriented_sobel = cv2.Sobel(S, cv2.CV_64F, orient=='x', orient=='y', ksize=sobel_kernel)
    abs_sobel = np.absolute(oriented_sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel < thresh[1])] = 1
    
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    
    S = image[:,:,2]
    sobelx = cv2.Sobel(S, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(S, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    mag_sobel = np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*mag_sobel/np.max(mag_sobel))
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    
    return mag_binary

def dir_thresh(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    
    S = image[:,:,2]
    sobelx = cv2.Sobel(S, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(S, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    direction = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary = np.zeros_like(direction)
    dir_binary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    
    return dir_binary

def sat_thresh(image, thresh=(0,255)):
    # Seperate saturation channel of HLS
    # Apply threshold
    
    S = image[:,:,2]
    sat_binary = np.zeros_like(S)
    sat_binary[(S >= thresh[0]) & (S < thresh[1])] = 1
    
    return sat_binary

def thresholded_image(image):
    # Choose a Sobel kernel size
    ksize = 5

    # Apply each of the thresholding functions
    gradx = sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(30, 100))
    grady = sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(30, 100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(100, 150))
    sat_binary = sat_thresh(image, thresh=(220,255))

    combined = np.zeros_like(mag_binary)
    combined[(((gradx == 1) & (grady == 1)) | mag_binary == 1) | sat_binary == 1] = 1
    
    return combined

## 4. Apply perspective transform

In [4]:
def perspective(binary_image):
    
    img_size = binary_image.shape
    
    src = np.float32([[180,img_size[0]],[570,462],[710,462],[1100,img_size[0]]])
    dst = np.float32([[200,img_size[0]],[200,0],[img_size[1]-200,0],[img_size[1]-200,img_size[0]]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    warped = cv2.warpPerspective(binary_image, M, img_size[::-1], flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv

## 5. Detect lane pixels and polyniminal fit

In [5]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

In [6]:
def search_around_poly(binary_warped, left_fit, right_fit):

    # width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the area of search based on activated x-values
    # within the +/- margin of our polynomial function
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

## 6. Determine the curvature of the lane and vehicle position

In [7]:
def find_curvature(binary_warped, leftx, lefty, rightx, righty):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/980 # meters per pixel in x dimension

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    # Define y-value where we want radius of curvature
    y_eval = np.max(ploty)
    
    # Implement the calculation of R_curve (radius of curvature) 
    left_curverad = (1+(2*left_fit_cr[0]*y_eval*ym_per_pix+left_fit_cr[1])**2)**1.5/np.absolute(2*left_fit_cr[0])
    right_curverad = (1+(2*right_fit_cr[0]*y_eval*ym_per_pix+right_fit_cr[1])**2)**1.5/np.absolute(2*right_fit_cr[0])
    
    pos = ((np.min(leftx) + np.min(rightx))/2 - binary_warped.shape[1]/2)*xm_per_pix
    
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    return pos, left_fit, right_fit, left_fitx, right_fitx, left_curverad, right_curverad

## 7. Warp the detected lane boundaries back onto the original image

In [8]:
def warp_back(undist, warped, Minv, left_fitx, right_fitx):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return result

## 8. Output visual display of the lane boundaries

In [9]:
def display_lane(result, curverad, pos):
    text1 = "Radius of Curvature = " + str(np.int(curverad)) + "(m)"
    text2 = "Vehicle is " + str(np.absolute(pos))[:5] + "m " + ("left" if pos < 0 else "right") + " of center"
    processed_frame = np.copy(result)
    cv2.putText(processed_frame, text1, (100, 100), cv2.FONT_HERSHEY_DUPLEX, 1.7, (255, 255, 255), 2)
    cv2.putText(processed_frame, text2, (100, 150), cv2.FONT_HERSHEY_DUPLEX, 1.7, (255, 255, 255), 2)
    
    return processed_frame

## 9. Build video processing pipeline

In [10]:
def process_video(video_path):
    
    # read project_video.mp4 and its fps & size
    cap = cv2.VideoCapture(video_path)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out_video_path = "output_" + video_path
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter(out_video_path, fourcc, fps, size)
    
    success, frame = cap.read()
    n_frame = 1
    
    # create array to smooth the detected data
    N_smooth = 10
    smooth = np.array([[]])
    
    while success:
        
        # pipeline
        undist = dist_corr(frame, mtx, dist)
        binary_image = thresholded_image(undist)
        warped, M, Minv = perspective(binary_image)
        
        if n_frame == 1:
            leftx, lefty, rightx, righty = find_lane_pixels(warped) # search from scratch
        else:
            leftx, lefty, rightx, righty = search_around_poly(warped, left_fit, right_fit) # search around last detected poly
        
        pos, left_fit, right_fit, left_fitx, right_fitx, left_curverad, right_curverad = find_curvature(warped, leftx, lefty, rightx, righty)
        curverad = min(left_curverad, right_curverad)
        
        # draw line area and warp back to undisted image
        result = warp_back(undist, warped, Minv, left_fitx, right_fitx)
        
        # smoothing process
        if n_frame == 1:
            smooth = np.array([list(left_fit)+list(right_fit)+[pos]+[curverad]])
        elif n_frame <= N_smooth:
            smooth = np.concatenate((smooth,[list(left_fit)+list(right_fit)+[pos]+[curverad]]))  
        else:
            smooth = np.concatenate((smooth,[list(left_fit)+list(right_fit)+[pos]+[curverad]]))
            smooth = smooth[1:,:]
        
        smoothed = np.sum(smooth, axis = 0)/smooth.shape[0]
        smooth[-1,:] = smoothed
        left_fit, right_fit, pos, curverad = smoothed[:3], smoothed[3:6], smoothed[-2], smoothed[-1]
        
        # display the calculated R_curve and position on the warped back image
        processed_frame = display_lane(result, curverad, pos)
        out.write(processed_frame)
        
        n_frame += 1
        success, frame = cap.read()

    cap.release()
    out.release()

In [11]:
video_path = "project_video.mp4"
%time process_video(video_path)

Wall time: 3min 3s
